In [1]:
import torch
from torch import nn
import random
import pandas as pd
import numpy as np
import json
import math
import re
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split, Dataset
from torch.nn import functional as F
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
import time
from string import punctuation
import string
import argparse
from argparse import ArgumentParser
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

/opt/conda/lib/python3.8/site-packages/pandas/compat/_optional.py:161: UserWarning: Pandas requires version '2.7.1' or newer of 'numexpr' (version '2.7.0' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.8/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
asdiv = pd.read_csv("Asdiv.csv")

In [3]:
asdiv.head()

,ID,Grade,Source,Body,Question,Solution_Type,Answer,Formula
0,nluds-0001,1,http://www.k5learning.com,Seven red apples and two green apples are in t...,How many apples are in the basket?,Addition,9 (apples),7+2=9
1,nluds-0002,1,http://www.k5learning.com,Ellen has six more balls than Marin. Marin has...,How many balls does Ellen have?,Addition,15 (balls),6+9=15
2,nluds-0003,1,http://www.k5learning.com,Janet has nine oranges and Sharon has seven or...,How many oranges do Janet and Sharon have toge...,Addition,16 (oranges),9+7=16
3,nluds-0004,1,http://www.k5learning.com,Allan brought two balloons and Jake brought fo...,How many balloons did Allan and Jake have in t...,Addition,6 (balloons),2+4=6
4,nluds-0005,1,http://www.k5learning.com,Adam has five more apples than Jackie. Jackie ...,How many apples does Adam have?,Addition,14 (apples),5+9=14


In [4]:
asdiv = asdiv[[ "Body", "Question", "Formula", "Answer"]]
asdiv.shape

(2305, 4)

In [5]:
asdiv.rename(columns={
    'Body': 'context',
    'Question': 'question',
    'Answer': 'final answer'
}, inplace=True)

In [6]:
asdiv.head()

,context,question,Formula,final answer
0,Seven red apples and two green apples are in t...,How many apples are in the basket?,7+2=9,9 (apples)
1,Ellen has six more balls than Marin. Marin has...,How many balls does Ellen have?,6+9=15,15 (balls)
2,Janet has nine oranges and Sharon has seven or...,How many oranges do Janet and Sharon have toge...,9+7=16,16 (oranges)
3,Allan brought two balloons and Jake brought fo...,How many balloons did Allan and Jake have in t...,2+4=6,6 (balloons)
4,Adam has five more apples than Jackie. Jackie ...,How many apples does Adam have?,5+9=14,14 (apples)


In [7]:
asdiv['Numeric Answer'] = asdiv['final answer'].str.extract(r'(\d+)')

In [8]:
# Split the equations based on the "=" symbol
asdiv[['answer', 'Right Side']] = asdiv['Formula'].str.split('=', 1, expand=True)

# Drop the original "Equation" column if not needed
asdiv.drop(columns=['Formula', 'Right Side'], inplace=True)

In [9]:
#asdiv["answer"] = asdiv["Formula"] #+ " final_answer:" + asdiv['Numeric Answer']

In [10]:
asdiv.head()

,context,question,final answer,Numeric Answer,answer
0,Seven red apples and two green apples are in t...,How many apples are in the basket?,9 (apples),9,7+2
1,Ellen has six more balls than Marin. Marin has...,How many balls does Ellen have?,15 (balls),15,6+9
2,Janet has nine oranges and Sharon has seven or...,How many oranges do Janet and Sharon have toge...,16 (oranges),16,9+7
3,Allan brought two balloons and Jake brought fo...,How many balloons did Allan and Jake have in t...,6 (balloons),6,2+4
4,Adam has five more apples than Jackie. Jackie ...,How many apples does Adam have?,14 (apples),14,5+9


In [11]:
asdiv = asdiv[["context", "question", "answer"]]

In [12]:
asdiv.head()

,context,question,answer
0,Seven red apples and two green apples are in t...,How many apples are in the basket?,7+2
1,Ellen has six more balls than Marin. Marin has...,How many balls does Ellen have?,6+9
2,Janet has nine oranges and Sharon has seven or...,How many oranges do Janet and Sharon have toge...,9+7
3,Allan brought two balloons and Jake brought fo...,How many balloons did Allan and Jake have in t...,2+4
4,Adam has five more apples than Jackie. Jackie ...,How many apples does Adam have?,5+9


In [13]:
asdiv.dtypes

context     object
question    object
answer      object
dtype: object

In [18]:
'''
# Function to replace numbers in the 'number' column based on the 'equation' column
def replace_numbers(row):
    numbers = row['Numbers']
    question = row["Body"]
    for i, num in enumerate(numbers):
        question = question.replace(f'number{i}', str(num))    

    return question

# Apply the functions to the DataFrame
svamp_testing['Body'] = svamp_testing.apply(replace_numbers, axis=1)

#print(svamp_testing)
'''

'\n# Function to replace numbers in the \'number\' column based on the \'equation\' column\ndef replace_numbers(row):\n    numbers = row[\'Numbers\']\n    question = row["Body"]\n    for i, num in enumerate(numbers):\n        question = question.replace(f\'number{i}\', str(num))    \n\n    return question\n\n# Apply the functions to the DataFrame\nsvamp_testing[\'Body\'] = svamp_testing.apply(replace_numbers, axis=1)\n\n#print(svamp_testing)\n'

In [19]:
'''
def swap_operator_with_next_value(equation_list):
    swapped_equation = ""
    i = 0
    while i < len(equation_list):
        if i + 1 < len(equation_list) and equation_list[i] in ["*", "/", "-", "+"]:
            swapped_equation += f"{equation_list[i+1]} {equation_list[i]} "
            i += 1
        else:
            swapped_equation += f"{equation_list[i]} "
        i += 1
    return swapped_equation.strip()

# Apply the function to swap operators in the 'Equation' column
svamp_testing['formula'] = svamp_testing['Equation'].apply(swap_operator_with_next_value)
'''

'\ndef swap_operator_with_next_value(equation_list):\n    swapped_equation = ""\n    i = 0\n    while i < len(equation_list):\n        if i + 1 < len(equation_list) and equation_list[i] in ["*", "/", "-", "+"]:\n            swapped_equation += f"{equation_list[i+1]} {equation_list[i]} "\n            i += 1\n        else:\n            swapped_equation += f"{equation_list[i]} "\n        i += 1\n    return swapped_equation.strip()\n\n# Apply the function to swap operators in the \'Equation\' column\nsvamp_testing[\'formula\'] = svamp_testing[\'Equation\'].apply(swap_operator_with_next_value)\n'

In [20]:
'''
# Function to replace numbers in the 'number' column based on the 'equation' column
def replace_numbers(row):
    numbers = row['Numbers']
    formula = row["formula"]
    for i, num in enumerate(numbers):
        formula = formula.replace(f'number{i}', str(num))    

    return formula

# Apply the functions to the DataFrame
svamp_testing['formula'] = svamp_testing.apply(replace_numbers, axis=1)
'''

'\n# Function to replace numbers in the \'number\' column based on the \'equation\' column\ndef replace_numbers(row):\n    numbers = row[\'Numbers\']\n    formula = row["formula"]\n    for i, num in enumerate(numbers):\n        formula = formula.replace(f\'number{i}\', str(num))    \n\n    return formula\n\n# Apply the functions to the DataFrame\nsvamp_testing[\'formula\'] = svamp_testing.apply(replace_numbers, axis=1)\n'

In [21]:
# Set the random seed for reproducibility
random_seed = 42

# Define the proportion of data to be used for validation
validation_ratio = 0.20

# Randomly sample the DataFrame to create the validation set
val_data = asdiv.sample(frac=validation_ratio, random_state=random_seed)

# Create the train set by excluding the validation set
train_data = asdiv.drop(val_data.index)

# Print the shapes of train and validation sets
print("Train set shape:", train_data.shape)
print("Validation set shape:", val_data.shape)

Train set shape: (1844, 3)
Validation set shape: (461, 3)


In [22]:
pretrained_model_path = 'pretrained_ce'
model = T5ForConditionalGeneration.from_pretrained(pretrained_model_path)
tokenizer = T5Tokenizer.from_pretrained(pretrained_model_path)

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [23]:
class GSMData(Dataset):
    def __init__(self, df, tokenizer, input_max_length=400, output_max_length=100):
        self.dataset = df
        self.tokenizer = tokenizer
        self.input_length = input_max_length
        self.output_length = output_max_length
    
    def __len__(self):
        return len(self.dataset)
    
    
    def __getitem__(self, index):
        
        data_row = self.dataset.iloc[index]
        source_encoding = tokenizer(
            data_row["question"],
            data_row["context"],
            max_length=self.input_length,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )
        
        target_encoding = tokenizer(
            data_row["answer"],
            max_length=self.output_length,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        
        labels = target_encoding["input_ids"]
        labels[labels == 0] = -100
        
        return dict(
            question=data_row["question"],
            context=data_row["context"],
            answer=data_row["answer"],
            input_ids=source_encoding["input_ids"].flatten(),
            attention_mask=source_encoding["attention_mask"].flatten(),
            labels=labels.flatten()
        )

In [24]:
class GSMDataModule(pl.LightningDataModule):
    def __init__(self, train_file, val_file, test_file, tokenizer_name_or_path, input_length=400, output_length=100, batch_size=8, num_workers=4):
        super().__init__()
        self.train_file = train_file
        self.val_file = val_file
        self.test_file = test_file
        self.tokenizer = T5Tokenizer.from_pretrained(tokenizer_name_or_path)
        self.input_length = input_length
        self.output_length = output_length
        self.batch_size = batch_size
        self.num_workers = num_workers
             
    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.train_data = GSMData(self.train_file, self.tokenizer)
            self.val_data = GSMData(self.val_file, self.tokenizer)
        if stage == 'test' or stage is None:
            self.test_data = GSMData(self.test_file, self.tokenizer)
        
    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, num_workers=self.num_workers)
     
    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size, num_workers=self.num_workers)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=self.batch_size, num_workers=self.num_workers)

In [25]:
test_data = val_data
data_module = GSMDataModule(train_data, val_data, test_data, pretrained_model_path)
data_module.setup()

In [26]:
class GSMModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(pretrained_model_path, return_dict = True)
        
    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(
            input_ids = input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return output.loss, output.logits
    
    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4)
        
        # Define the ReduceLROnPlateau scheduler
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
        
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss',  # Monitor validation loss for scheduler
                'interval': 'epoch',    # Adjust learning rate after each epoch
                'frequency': 1          # Frequency to reduce LR (every epoch in this case)
            }
        }

In [27]:
model = GSMModel()

In [28]:
checkpoint_callback = ModelCheckpoint(
    dirpath="Finetuned_checkpoint",
    filename='best_model_ce_asdiv_mask_50_span_3',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min',
    save_weights_only=True,
)

In [29]:
# create a logger instance
logger = TensorBoardLogger(save_dir='logs/', name='Asdiv_Finetuning_Logs')

In [30]:
trainer = pl.Trainer(
    callbacks=[checkpoint_callback],
    max_epochs = 20,
    logger=logger
)

/opt/conda/lib/python3.8/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [31]:
trainer.fit(model,data_module)

/home/work/.local/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /home/work/GSM8K/CustomT5/Finetuned_checkpoint exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 783 M 
-----------------------------------------------------
783 M     Trainable params
0         Non-trainable params
783 M     Total params
3,132.600 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Epoch 18, global step 4389: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 19, global step 4620: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 00020: reducing learning rate of group 0 to 1.0000e-08.


In [32]:
### predictions
finetuned_model = GSMModel.load_from_checkpoint("Finetuned_checkpoint/best_model_ce_asdiv_mask_50_span_3.ckpt")
finetuned_model.freeze()
#finetuned_model.eval()

In [33]:
def generate_answer(question, return_sequences):
    source_encoding = tokenizer(
        question["question"],
        question["context"],
        max_length=400,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    # Move the input tensors to the same device as the model
    input_ids = source_encoding["input_ids"].to(finetuned_model.device)
    attention_mask = source_encoding["attention_mask"].to(finetuned_model.device)

    generated_ids = finetuned_model.model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        num_beams=80,
        max_length=100,
        early_stopping=True,
        use_cache=True,
        repetition_penalty=3.0,
        #temperature=1,
        num_return_sequences = return_sequences
    )

    preds = [tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_space=True)
             for gen_id in generated_ids]

    return preds#" ".join(preds)

In [34]:
# Assuming you have the tokenizer and finetuned_model already defined and initialized
# Question and Context data
question_data = {
    "question": "How many apples are in the basket?",
    "context": "Seven red apples and two green apples are in the basket."
}

# Get the output
return_sequences = 2  # Set to True if you want multiple generated sequences
output = generate_answer(question_data, return_sequences)
print(output)  # This will print the generated answer


['7+2', '7+2']


In [35]:
svamp_data = pd.read_json("SVAMP.json")
svamp_data.head()

,ID,Body,Question,Equation,Answer,Type
0,chal-1,Each pack of dvds costs 76 dollars. If there i...,How much do you have to pay to buy each pack?,( 76.0 - 25.0 ),51,Subtraction
1,chal-2,Dan had $ 3 left with him after he bought a ca...,How much did the candy bar cost?,( 4.0 - 3.0 ),1,Subtraction
2,chal-3,Paco had 26 salty cookies and 17 sweet cookies...,How many salty cookies did Paco have left?,( 26.0 - 9.0 ),17,Subtraction
3,chal-4,43 children were riding on the bus. At the bus...,How many children got off the bus at the bus s...,( 43.0 - 21.0 ),22,Subtraction
4,chal-5,28 children were riding on the bus. At the bus...,How many more children got on the bus than tho...,( 30.0 - 28.0 ),2,Subtraction


In [36]:
svamp_data.rename(columns={
    'Body': 'context',
    'Question': 'question',
    'Answer': 'answer'
}, inplace=True)

In [37]:
svamp_data = svamp_data[['context', 'question', 'answer', "Equation"]]
svamp_data.shape

(1000, 4)

In [38]:
import warnings

# Use the filterwarnings function to suppress SyntaxWarning
warnings.filterwarnings("ignore", category=SyntaxWarning)

pass_outcome = {}
temp_list = []
# Generate sequences for the largest value in pass_list
max_return_seq = 50 #max(pass_list)
sequences = {}  # Dictionary to store the generated sequences for each question
answers = {}
for i in range(len(svamp_data)):
    question = svamp_data.iloc[i]
    sequences[i] = generate_answer(question, max_return_seq)
    temp_list = []
    for j in range(len(sequences[i])):
        try:
            temp_list.append(eval(sequences[i][j]))            
        except:
            temp_list.append("Not executable")
    answers[i] = temp_list
    
print("sequence generation complete")

sequence generation complete


In [39]:
#sequences, answers

In [40]:
sequences1 = sequences
answers1 = answers

In [41]:
svamp_data.head()

,context,question,answer,Equation
0,Each pack of dvds costs 76 dollars. If there i...,How much do you have to pay to buy each pack?,51,( 76.0 - 25.0 )
1,Dan had $ 3 left with him after he bought a ca...,How much did the candy bar cost?,1,( 4.0 - 3.0 )
2,Paco had 26 salty cookies and 17 sweet cookies...,How many salty cookies did Paco have left?,17,( 26.0 - 9.0 )
3,43 children were riding on the bus. At the bus...,How many children got off the bus at the bus s...,22,( 43.0 - 21.0 )
4,28 children were riding on the bus. At the bus...,How many more children got on the bus than tho...,2,( 30.0 - 28.0 )


In [42]:
sequences1 = pd.DataFrame.from_dict(sequences1, orient='index')
answers1 = pd.DataFrame.from_dict(answers1, orient='index')
sequences1.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,76-25,76-25,76-25,76/25,4*76-25,75*4-25,76-75,75/25,6*4-25,76-35,...,76-20.5,56-25,75-35,276-25,6*5,56*5,76-24,78-25,760-25,*76-25
1,4-3,(4-3)*4,4+3,3+4,(4-5)-3,(4-3)*2,(4-5)*4,"GCD(3,4)",(3-2)*4,(4-3)*4,...,(3-2)-3,(3/3)*4,(4-0)*4,4:13,(3-2)-4,(4-5)/3,-4+3,(3/4)-3,(3-5)-4,4:14+3
2,26-9,26+17-9,26-4,26-9,6+17-9,27-9,26-27-9,26+9,26-9,6+14-9,...,26+14-9,16-9,25-9,26-3,26-7-9,+26-9,26- 9,46-9,; 26-9,36-9
3,43-21,43+21,4321-21,43-21,43 21,43-21,42-21,43 + 21,47+21,43-21,...,43 21-22,43+-21,4321-20,38+21,43-7-21,43-19,43-21,46-21,34+21,43-9-21
4,30-28,30-82,30-30-28,30-30,30-30-82,30 28-82,30-30+28,30-30 28,30+28,30-28,...,35-30-28,"30, 28-82",30-2,-30-82,30-30- 28,28+82,31-28,30-84,30 -28,30-30-21


In [43]:
# You can create a dictionary to map old column names to new column names
column_mapping = {i: f"equation{i}" for i in range(max_return_seq+1)}

# Use the 'rename' method with the 'columns' parameter to rename the columns
sequences1.rename(columns=column_mapping, inplace=True)

In [44]:
# You can create a dictionary to map old column names to new column names
column_mapping = {i: f"solution{i}" for i in range(max_return_seq+1)}

# Use the 'rename' method with the 'columns' parameter to rename the columns
answers1.rename(columns=column_mapping, inplace=True)

In [45]:
# Concatenate them side by side
concatenated_df = pd.concat([svamp_data, sequences1, answers1], axis=1)
concatenated_df.head()

,context,question,answer,Equation,equation0,equation1,equation2,equation3,equation4,equation5,...,solution40,solution41,solution42,solution43,solution44,solution45,solution46,solution47,solution48,solution49
0,Each pack of dvds costs 76 dollars. If there i...,How much do you have to pay to buy each pack?,51,( 76.0 - 25.0 ),76-25,76-25,76-25,76/25,4*76-25,75*4-25,...,55.5,31,40,251,30,280,52,53,735,Not executable
1,Dan had $ 3 left with him after he bought a ca...,How much did the candy bar cost?,1,( 4.0 - 3.0 ),4-3,(4-3)*4,4+3,3+4,(4-5)-3,(4-3)*2,...,-2,4.0,16,Not executable,-3,-0.333333,-1,-2.25,-6,Not executable
2,Paco had 26 salty cookies and 17 sweet cookies...,How many salty cookies did Paco have left?,17,( 26.0 - 9.0 ),26-9,26+17-9,26-4,26-9,6+17-9,27-9,...,31,7,16,23,10,17,17,37,Not executable,27
3,43 children were riding on the bus. At the bus...,How many children got off the bus at the bus s...,22,( 43.0 - 21.0 ),43-21,43+21,4321-21,43-21,43 21,43-21,...,Not executable,22,4301,59,15,24,22,25,55,13
4,28 children were riding on the bus. At the bus...,How many more children got on the bus than tho...,2,( 30.0 - 28.0 ),30-28,30-82,30-30-28,30-30,30-30-82,30 28-82,...,-23,"(30, -54)",28,-112,-28,110,3,-54,2,-21


In [46]:
concatenated_df.to_csv('predictions_ce_svamp_mask_50_span_3.csv', index=False)

### We can use the same finetuned model for multiarith dataset as well and check the performance

In [47]:
mutiarith_data = pd.read_json("mutiarith.json")

In [48]:
mutiarith_data.shape

(1492, 7)

In [49]:
mutiarith_data.head()

,iIndex,sQuestion,quants,lAlignments,lEquations,lSolutions,rates
0,1,Joan found 70.0 seashells on the beach . She g...,"[70.0, 27.0]","[0, 1]",[X=(70.0-27.0)],[43.0],[]
1,2,There were 28.0 bales of hay in the barn . Tim...,"[28.0, 54.0]","[1, 0]",[X=(54.0-28.0)],[26.0],[]
2,3,Mary is baking a cake . The recipe wants 8.0 c...,"[8.0, 2.0]","[0, 1]",[X=(8.0-2.0)],[6.0],[]
3,4,Sara 's high school played 12.0 basketball gam...,"[12.0, 4.0]","[0, 1]",[X=(12.0-4.0)],[8.0],[]
4,5,There are 22.0 walnut trees currently in the p...,"[22.0, 55.0]","[1, 0]",[X=(55.0-22.0)],[33.0],[]


In [50]:
mutiarith_test = mutiarith_data[["sQuestion", "lEquations", "lSolutions"]]

In [51]:
mutiarith_test.rename(columns={
    'lSolutions': 'answer'
}, inplace=True)

/tmp/ipykernel_143/3878747415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mutiarith_test.rename(columns={


In [52]:
mutiarith_test.head()

,sQuestion,lEquations,answer
0,Joan found 70.0 seashells on the beach . She g...,[X=(70.0-27.0)],[43.0]
1,There were 28.0 bales of hay in the barn . Tim...,[X=(54.0-28.0)],[26.0]
2,Mary is baking a cake . The recipe wants 8.0 c...,[X=(8.0-2.0)],[6.0]
3,Sara 's high school played 12.0 basketball gam...,[X=(12.0-4.0)],[8.0]
4,There are 22.0 walnut trees currently in the p...,[X=(55.0-22.0)],[33.0]


In [53]:
# Define a function to split the sentence into question and context
def split_sentence(row):
    sentence = row["sQuestion"]
    sentences = sentence.split(". ")
    question = sentences[-1]
    context = ". ".join(sentences[:-1])
    return pd.Series({"question": question, "context": context})

# Apply the split_sentence function to the DataFrame
mutiarith_test[["question", "context"]] = mutiarith_test.apply(lambda row: split_sentence(row), axis=1)

# Drop the original "sentence" column if needed
test_set = mutiarith_test.drop(columns=["sQuestion"])

/tmp/ipykernel_143/3076847272.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mutiarith_test[["question", "context"]] = mutiarith_test.apply(lambda row: split_sentence(row), axis=1)
/tmp/ipykernel_143/3076847272.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mutiarith_test[["question", "context"]] = mutiarith_test.apply(lambda row: split_sentence(row), axis=1)


In [54]:
test_set.head()

,lEquations,answer,question,context
0,[X=(70.0-27.0)],[43.0],How many seashells did she give to Sam ?,Joan found 70.0 seashells on the beach . She g...
1,[X=(54.0-28.0)],[26.0],How many bales did he store in the barn ?,There were 28.0 bales of hay in the barn . Tim...
2,[X=(8.0-2.0)],[6.0],How many cups does she need to add ?,Mary is baking a cake . The recipe wants 8.0 c...
3,[X=(12.0-4.0)],[8.0],How many games did they win ?,Sara 's high school played 12.0 basketball gam...
4,[X=(55.0-22.0)],[33.0],How many walnut trees did the workers plant to...,There are 22.0 walnut trees currently in the p...


In [55]:
def generate_answer(question, return_sequences):
    source_encoding = tokenizer(
        question["question"],
        question["context"],
        max_length=400,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    # Move the input tensors to the same device as the model
    input_ids = source_encoding["input_ids"].to(finetuned_model.device)
    attention_mask = source_encoding["attention_mask"].to(finetuned_model.device)

    generated_ids = finetuned_model.model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        num_beams=80,
        max_length=100,
        early_stopping=True,
        use_cache=True,
        #repetition_penalty=2.0,
        #temperature=0.8,
        num_return_sequences = return_sequences
    )

    preds = [tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_space=True)
             for gen_id in generated_ids]

    return preds#" ".join(preds)

In [56]:
# Assuming you have the tokenizer and finetuned_model already defined and initialized
# Question and Context data
question_data = {
    "question": "How many seashells did she give to Sam ?",
    "context": "Joan found 70.0 seashells on the beach . She gave Sam some of her seashells . She has 27.0 seashells "
}

# Get the output
return_sequences = 2  # Set to True if you want multiple generated sequences
output = generate_answer(question_data, return_sequences)
print(output)  # This will print the generated answer


['70.0-27.0', '70-27.0']


In [57]:
import warnings

# Use the filterwarnings function to suppress SyntaxWarning
warnings.filterwarnings("ignore", category=SyntaxWarning)

pass_outcome = {}
temp_list = []
# Generate sequences for the largest value in pass_list
max_return_seq = 50 #max(pass_list)
sequences = {}  # Dictionary to store the generated sequences for each question
answers = {}
for i in range(len(test_set)):
    question = test_set.iloc[i]
    sequences[i] = generate_answer(question, max_return_seq)
    temp_list = []
    for j in range(len(sequences[i])):
        try:
            temp_list.append(eval(sequences[i][j]))            
        except:
            temp_list.append("Not executable")
    answers[i] = temp_list
    
print("sequence generation complete")

sequence generation complete


In [58]:
sequences2 = pd.DataFrame.from_dict(sequences, orient='index')
answers2 = pd.DataFrame.from_dict(answers, orient='index')
sequences2.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,70.0-27.0,70-27.0,70-20-27.0,70+27.0,700.2-27.0,700-0-27.0,700.0-27.0,70-27,70.2-27.0,70.0-27,...,70.0-27.,70.0-28.0,70.00-27.0,70+ 27.0,70-20-27,700-27.7,700.0-27,10.0-27.0,1.0-27,70.0-25.8
1,54.0-28,54.0-28.0,534.0-28,554.0-28,524.0-28,534.0-28.0,54.0-25.8,54.0-28,504.0-28,53.40-28,...,534.0-25.8,54.0-28,-54.0-28,524.0-28.0,(54.0-28),58.0-28,54.0-28.0,528.0-28,54.0 28.8,514.0-28.0
2,8.0-2,8.0-2.0,8.0-2.0,8-2,8.0-0.2,8.0-2,(8.0-2),8.0-0.2,8.0+2.0,8000-2.0,...,the 8.0-2,0.8-2,8.0-2,8.0+2.0,8.0-.2,8.0-2.5,of 8.0-2,", 8.0-2",8.0- 2.0,14.0-2
3,12.0-4,12.0-4.0,12.0+4.0,12.0-4.0,124.0-4,12+4.0,124.0-4.0,120-4.0,12-4,12.0-3,...,120.4+4.0,12.20-4,120.2-4.0,1240-4.0,12-20,12.2-4,12-40,1240-4,120-0.4,14.0-4.0
4,55.0-22.0,55.0-22,55.0-22.0,55.0-22.20,55.0-20.2,55.0-220,55.0-24.0,55.0-22.2,55.0- 22.0,55.0-22.8,...,55.0- 220,55.0-22+2,55.0-20.2,550- 22.0,55.00-22.0,55.0-22,55.0-22-0,55.0-22.0,55.0-220.2,22.0+55.0


In [59]:
# You can create a dictionary to map old column names to new column names
column_mapping = {i: f"equation{i}" for i in range(max_return_seq+1)}

# Use the 'rename' method with the 'columns' parameter to rename the columns
sequences2.rename(columns=column_mapping, inplace=True)

In [60]:
# You can create a dictionary to map old column names to new column names
column_mapping = {i: f"solution{i}" for i in range(max_return_seq+1)}

# Use the 'rename' method with the 'columns' parameter to rename the columns
answers2.rename(columns=column_mapping, inplace=True)

In [61]:
# Concatenate them side by side
concatenated_df = pd.concat([test_set, sequences2, answers2], axis=1)
concatenated_df.head()

,lEquations,answer,question,context,equation0,equation1,equation2,equation3,equation4,equation5,...,solution40,solution41,solution42,solution43,solution44,solution45,solution46,solution47,solution48,solution49
0,[X=(70.0-27.0)],[43.0],How many seashells did she give to Sam ?,Joan found 70.0 seashells on the beach . She g...,70.0-27.0,70-27.0,70-20-27.0,70+27.0,700.2-27.0,700-0-27.0,...,43.0,42.0,43.0,97.0,23,672.3,673.0,-17.0,-26.0,44.2
1,[X=(54.0-28.0)],[26.0],How many bales did he store in the barn ?,There were 28.0 bales of hay in the barn . Tim...,54.0-28,54.0-28.0,534.0-28,554.0-28,524.0-28,534.0-28.0,...,508.2,26.0,-82.0,496.0,26.0,30.0,26.0,500.0,Not executable,486.0
2,[X=(8.0-2.0)],[6.0],How many cups does she need to add ?,Mary is baking a cake . The recipe wants 8.0 c...,8.0-2,8.0-2.0,8.0-2.0,8-2,8.0-0.2,8.0-2,...,Not executable,-1.2,6.0,10.0,7.8,5.5,Not executable,Not executable,6.0,12.0
3,[X=(12.0-4.0)],[8.0],How many games did they win ?,Sara 's high school played 12.0 basketball gam...,12.0-4,12.0-4.0,12.0+4.0,12.0-4.0,124.0-4,12+4.0,...,124.4,8.2,116.2,1236.0,-8,8.2,-28,1236,119.6,10.0
4,[X=(55.0-22.0)],[33.0],How many walnut trees did the workers plant to...,There are 22.0 walnut trees currently in the p...,55.0-22.0,55.0-22,55.0-22.0,55.0-22.20,55.0-20.2,55.0-220,...,-165.0,35.0,34.8,528.0,33.0,33.0,33.0,33.0,-165.2,77.0


In [62]:
concatenated_df.to_csv('predictions_ce_multiarith_mask_50_span_3.csv', index=False)